# Implementing VGG19 paper from scratch in Keras

In [1]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Activation, Dropout, Flatten, Dense

import numpy as np
np.random.seed(42)

## Creating the model

In [2]:
input_shape = (224,224,3)
initializer = tf.keras.initializers.GlorotNormal() # the authors of VGG19 paper did not used Glorot Xavier initialization but they mentioned that this can be also used

VGG19 = Sequential([
    Conv2D(filters = 64, kernel_size=(3,3), padding="same", activation='relu', input_shape=input_shape, kernel_initializer = initializer),
    Conv2D(filters = 64, kernel_size=(3,3), padding="same", activation='relu', kernel_initializer = initializer),
    MaxPool2D(pool_size=(2,2), strides=(2,2)),
    
    Conv2D(filters = 128, kernel_size=(3,3), padding="same", activation='relu', kernel_initializer = initializer),
    Conv2D(filters = 128, kernel_size=(3,3), padding="same", activation='relu', kernel_initializer = initializer),
    MaxPool2D(pool_size=(2,2), strides=(2,2)),
    
    Conv2D(filters = 256, kernel_size=(3,3), padding="same", activation='relu'),
    Conv2D(filters = 256, kernel_size=(3,3), padding="same", activation='relu'),
    Conv2D(filters = 256, kernel_size=(3,3), padding="same", activation='relu'),
    Conv2D(filters = 256, kernel_size=(3,3), padding="same", activation='relu'),
    MaxPool2D(pool_size=(2,2), strides=(2,2)),
    
    Conv2D(filters = 512, kernel_size=(3,3), padding="same", activation='relu'),
    Conv2D(filters = 512, kernel_size=(3,3), padding="same", activation='relu'),
    Conv2D(filters = 512, kernel_size=(3,3), padding="same", activation='relu'),
    Conv2D(filters = 512, kernel_size=(3,3), padding="same", activation='relu'),
    MaxPool2D(pool_size=(2,2), strides=(2,2)),
    
    Conv2D(filters = 512, kernel_size=(3,3), padding="same", activation='relu'),
    Conv2D(filters = 512, kernel_size=(3,3), padding="same", activation='relu'),
    Conv2D(filters = 512, kernel_size=(3,3), padding="same", activation='relu'),
    Conv2D(filters = 512, kernel_size=(3,3), padding="same", activation='relu'),
    MaxPool2D(pool_size=(2,2), strides=(2,2)),
    
    Flatten(),
    Dense(units=4096, activation='relu', kernel_regularizer='l2', kernel_initializer = initializer),
    Dropout(0.5),
    Dense(units=4096, activation='relu', kernel_regularizer='l2', kernel_initializer = initializer),
    Dropout(0.5),
    Dense(units=1000, activation='softmax', kernel_initializer = initializer)

])

VGG19.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 256)       2

In [3]:
VGG19.save("../working/VGG19Arch.keras")

## Data Preparation

##### VGG19 uses random cropping and random horizontal flip for data augmentation
Since random cropping is not available in ImageDataGenerator, we create our own random crop generator

In [8]:
def random_crop(img, random_crop_size):
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]

def crop_generator(batches, crop_length):
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3));
        for i in range(batch_x.shape[0]):
            batch_crops = random_crop(batch_x[i], (crop_length, crop_length))
        yield(batch_crops, batch_y)

In [13]:
from keras.preprocessing.image import ImageDataGenerator

TRAIN_DIR = "../input/imagenetmini-1000/imagenet-mini/train/"
VAL_DIR = "../input/imagenetmini-1000/imagenet-mini/val/"
batch_size = 64

train_generator = ImageDataGenerator(horizontal_flip = True)
val_generator = ImageDataGenerator(horizontal_flip = True)

train_gen = train_generator.flow_from_directory(TRAIN_DIR, target_size=(256, 256), batch_size=batch_size, class_mode='categorical')
val_gen = val_generator.flow_from_directory(VAL_DIR, target_size=(256, 256), batch_size=batch_size, class_mode='categorical')

train_batches = crop_generator(train_gen, 224)
val_batches = crop_generator(val_gen, 224)

Found 34745 images belonging to 1000 classes.
Found 3923 images belonging to 1000 classes.


## Training the model

In [14]:
VGG19.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 2

history = VGG19.fit_generator(train_batches, 
                              epochs = epochs, 
                              steps_per_epoch = 34745 // batch_size, 
                              validation_data = val_batches,
                              validation_steps = 3923 // batch_size,
                              verbose = 1)

Epoch 1/2
395/542 [====================>.........] - ETA: 2:35 - loss: 29.9660 - accuracy: 0.0026